# TICKER TECHNICAL ANALYSIS

<img src="media/bull_bear.jpeg" alt="Alt Text" width="150"/>

Analsys of buy and sell signals for a given ticker price action for the specified date range. The buy and sell 
signals are based on common technical indicators, custom algorythms & compared against Machine Learning models.

### Imported Modules

In [270]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

### Date Range Ticker


In [271]:
"""
Get current date so we can always get the most recent date from yahoo finance.
date is formated at as YY-mm-dd.

"""


date = datetime.now() + timedelta(days=1)
end_date_fmt = date.strftime("%Y-%m-%d")
start_date = '2022-01-01'
ticker = 'sofi'

### Yahoo API Using Date Range and Ticker

In [272]:
df_daily = yf.download(ticker, start_date, end_date_fmt, progress=False)

In [273]:
"""
The date_range function filters the dataframe for exact date range or span.

"""

def date_range(data, start=None, end=None, span=None):
    if start and end:
        return data[start:end]
    elif span is not None:
        cutoff_date = pd.Timestamp.now()-pd.Timedelta(days=span)
        return data[df_daily.index > cutoff_date]
    else:
        raise ValueError("Please provide either a date range (start='yyyy-mm-dd', end='yyyy-mm-dd') or span")



### Technical Indicators

EMA

In [274]:
""""
The EMA function calculates the Exponential Moving Average (ema) by finding the average between the fast and slow ema.
The Fastema and Slowema are added as columns to the database.

"""

def ema(dataset, fast, slow):
   dataset['Fastema'] = dataset['Close'].ewm(span=fast, adjust=False).mean()
   dataset['Slowema'] = dataset['Close'].ewm(span=slow, adjust=False).mean()
   return dataset

MACD


In [275]:
"""
The macd1 function is calcuating the Moving Average Convergence Divergence (MACD) by finding the difference between 12 and 26 EMA.
Generate singals based on 9 day moving average.
Inserting a macd histogram that shows macd and signal crossing by finding the difference.
"""


def macd1(dataset):
    dataset['macd12'] =  dataset['Close'].ewm(span=12, adjust=False).mean()
    dataset['macd26'] = dataset['Close'].ewm(span=26, adjust=False).mean()
    dataset['macd'] = dataset['macd12'] - dataset['macd26']
    dataset['signal'] = dataset['macd'].ewm(span=9, adjust=False).mean()
    dataset['macd_hist'] = dataset['macd'] - dataset['signal']
    return dataset





RSI

In [276]:
"""
The RSI function calculates the Relative Strength Index (RSI).
The Median_RSI function calculates overbought_level and oversold_level to find the average.

"""

def rsi(dataset, overbought_level, oversold_level):
    delta = dataset['Close'].diff(1)

    # Calculate the exponentially weighted moving averages of gains and losses
    gain_ewm = delta.where(delta > 0, 0).ewm(span=14, adjust=False).mean()
    loss_ewm = -delta.where(delta < 0, 0).ewm(span=14, adjust=False).mean()

    # Calculate the relative strength (RS)
    rs = gain_ewm / loss_ewm

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))
    dataset['RSI'] = rsi

    median_rsi = np.median([overbought_level, oversold_level])
    dataset['Median_RSI'] = np.where(rsi > median_rsi, 1, -1)

    return(dataset)


In [277]:
# closing prices is higher than the opening price - bullish using 1
# closing prices is lower than the opening price - bearish using -1
# Closing prices and opening price are equal = unchange using 0

In [ ]:
'''
The Candle Pattern function is adding the column Candle_Pattern to the dataframe
Candle_Pattern is represented by 0 meaing no change, 1 meaning Close is > the Open, -1 Close < the Open indicating which candle to show.
'''

def candle_pattern(dataset):
    dataset['Candle_Pattern'] = 0
    dataset.loc[dataset['Close'] > dataset['Open'], 'Candle_Pattern'] = 1
    dataset.loc[dataset['Close'] < dataset['Open'], 'Candle_Pattern'] = -1
    return dataset




In [279]:
candl = candle_pattern(df_daily)
candl.tail()

,Open,High,Low,Close,Adj Close,Volume,Candle_Pattern
Date,,,,,,,
2024-11-01,11.21,11.220,10.810,11.04,11.04,41742300,-1
2024-11-04,10.88,11.130,10.631,10.91,10.91,41321700,1
2024-11-05,10.96,11.550,10.950,11.42,11.42,53857800,1
2024-11-06,12.07,12.195,11.630,11.81,11.81,63197700,-1
2024-11-07,11.91,12.140,11.850,11.90,11.90,54208761,-1


### Calculating Combined Technical Indicators

In [280]:
# Added a variable called all_indicators that chains/links all technical indicators functions to run all at once.
all_indicators = rsi(macd1(ema(df_daily,5, 7)),70, 30)

In [281]:
# Shows the first 5 rows of all_indicatos dataset.
all_indicators.tail()

,Open,High,Low,Close,Adj Close,Volume,Candle_Pattern,Fastema,Slowema,macd12,macd26,macd,signal,macd_hist,RSI,Median_RSI
Date,,,,,,,,,,,,,,,,
2024-11-01,11.21,11.220,10.810,11.04,11.04,41742300,-1,10.999909,10.913571,10.629208,9.860012,0.769196,0.740091,0.029105,63.979123,1
2024-11-04,10.88,11.130,10.631,10.91,10.91,41321700,1,10.969939,10.912678,10.672407,9.937788,0.734618,0.738997,-0.004378,59.892442,1
2024-11-05,10.96,11.550,10.950,11.42,11.42,53857800,1,11.119959,11.039509,10.787421,10.047582,0.739839,0.739165,0.000674,68.888109,1
2024-11-06,12.07,12.195,11.630,11.81,11.81,63197700,-1,11.349973,11.232132,10.944741,10.178131,0.766610,0.744654,0.021956,74.028012,1
2024-11-07,11.91,12.140,11.850,11.90,11.90,54208761,-1,11.533315,11.399099,11.091704,10.305677,0.786027,0.752929,0.033098,75.122369,1


In [ ]:
''''
The Technical_Algorithm function used to identify bullish and bearish trends in the dataset based on various technical indicators.
Variables are created to check for bullish and bearish mixtures.
The dictionary is used to call all mixtures.
Trends are to anaylize the bullish and bearish mixtures.

'''

def technical_algorithm(dataset, bullish_algo_mix, bearish_algo_mix):
    # This secetion is where I'll define the bullish mixtures.
    ema_bullish_cross = (dataset['Fastema'] > dataset['Slowema']) & (dataset['Fastema'] < dataset['Slowema']).shift(1)
    rsi_bullish_check = dataset['Median_RSI'] == 1
    macd_bullish_cross = dataset['macd12'] > dataset['macd26']
    macd_bullish_hist = dataset['macd_hist'] > dataset['macd_hist'].shift(1)
    candle_bullish = dataset.loc[dataset['Close'] > dataset['Open'], 'Candle_Pattern'] == 1
    # This secetion is where I'll define the bearish mixtures.
    ema_bearish_cross = (dataset['Fastema'] < dataset['Slowema']) & (dataset['Fastema'] >= dataset['Slowema']).shift(1)
    rsi_bearish_check = dataset['Median_RSI'] == -1
    macd_bearish_cross = dataset['macd12'] < dataset['macd26']
    macd_bearish_hist =  dataset['macd_hist'] < dataset['macd_hist'].shift(1)
    candle_bearish = dataset.loc[dataset['Close'] < dataset['Open'], 'Candle_Pattern'] == -1

    # This will be a dictionary of all my mixtures.
    algo_mix = {
        'bullish1': ema_bullish_cross,
        'bearish1': ema_bearish_cross,
        'bullish2': rsi_bullish_check,
        'bearish2': rsi_bearish_check,
        'bullish3': macd_bullish_cross,
        'bearish3': macd_bearish_cross,
        'bullish4': macd_bullish_hist,
        'bearish4': macd_bearish_hist,
        'bullish5': candle_bullish,
        'bearish5': candle_bearish
    }

    # Set our Bullish and bearish trends based on mixture.
    bullish_algo = algo_mix[bullish_algo_mix]
    bearish_algo = algo_mix[bearish_algo_mix]

    dataset['Trend'] = 0
    dataset.loc[bullish_algo, 'Trend'] = 1
    dataset.loc[bearish_algo, 'Trend'] = -1

    return dataset


In [283]:
tech_al = technical_algorithm(all_indicators, 'bullish4', 'bearish4')
tech_al.tail(20)

,Open,High,Low,Close,Adj Close,Volume,Candle_Pattern,Fastema,Slowema,macd12,macd26,macd,signal,macd_hist,RSI,Median_RSI,Trend
Date,,,,,,,,,,,,,,,,,
2024-10-11,8.65,9.070,8.630,9.01,9.01,42833300,1,8.622119,8.507372,8.314910,8.014790,0.300120,0.214297,0.085824,81.133192,1,1
2024-10-14,9.62,10.070,9.375,10.04,10.04,116449900,1,9.094746,8.890529,8.580309,8.164806,0.415503,0.254538,0.160965,89.820935,1,1
2024-10-15,10.16,10.490,10.030,10.18,10.18,87195400,1,9.456498,9.212897,8.826415,8.314079,0.512336,0.306098,0.206238,90.506538,1,1
2024-10-16,10.23,10.325,9.910,10.04,10.04,48045000,-1,9.650998,9.419673,9.013121,8.441925,0.571195,0.359117,0.212078,83.979916,1,1
2024-10-17,10.01,10.030,9.620,9.87,9.87,50293900,-1,9.723999,9.532254,9.144948,8.547709,0.597240,0.406742,0.190498,76.273538,1,-1
2024-10-18,9.91,10.180,9.881,10.18,10.18,31400300,1,9.875999,9.694191,9.304187,8.668619,0.635568,0.452507,0.183061,80.113258,1,-1
2024-10-21,10.26,10.540,10.190,10.40,10.40,50814300,1,10.050666,9.870643,9.472774,8.796869,0.675904,0.497186,0.178718,82.440236,1,-1
2024-10-22,10.32,10.620,10.320,10.58,10.58,36501700,1,10.227111,10.047982,9.643116,8.928953,0.714163,0.540582,0.173581,84.187031,1,-1
2024-10-23,10.55,10.650,10.175,10.42,10.42,38130400,-1,10.291407,10.140987,9.762637,9.039401,0.723236,0.577112,0.146123,76.392856,1,-1
